# Here, the codes for our method (AnyLoss) is provided

## 1. Code for Single-Layer Perceptron (SLP)

In [ ]:
################################# SLP with Mean Square Error (MSE) - Baseline 1 #################################
def linear(X,theta):
    z = np.dot(X,theta.T)
    return z

def cost_ada(z,y):
    loss = ((y-z)**2).sum()
    return loss/2.0

def gd_ada(X,z,y):
    return -np.dot((y-z), X)

def update_loss(theta,learning_rate,gradient):
    return theta-(learning_rate*gradient)

def predict(X,theta):
    outcome = []
    result = linear(X,theta)
    for i in range(X.shape[0]):
        if result[i] <= 0:
            outcome.append(-1)
        else:
            outcome.append(1)
    return outcome 


theta = np.zeros(X_train_a.shape[1])   # Initialize theta with zeros
num_iter = 1000
cost = []
lr = 0.1

for i in range(num_iter):
    z = linear(X_train_a,theta)
    cost.append(cost_ada(z,y_train))
    gradient = gd_ada(X_train_a,z,y_train)
    theta = update_loss(theta,lr,gradient)
outcome_r = predict(X_train_a,theta)

In [ ]:
################################# SLP with Binary Cross-Entropy (BCE) - Baseline 2 #################################
def sigmoid(X,theta):
    z = np.dot(X,theta.T).astype(float)
    return 1.0/(1+np.exp(-z))

def cost_function(h,y):
    loss = ((-y * np.log(h))-((1-y)* np.log(1-h))).mean()
    return loss

def gradient_descent(X,h,y,yl):
    return np.dot(X.T,(h-y))/yl 

def update_loss(theta,learning_rate,gradient):
    return theta-(learning_rate*gradient)

def predict(X,theta):
    outcome = []
    result = sigmoid(X,theta)
    for i in range(X.shape[0]):
        if result[i] <= threshold:
            outcome.append(0)
        else:
            outcome.append(1)
    return outcome  


theta = np.zeros(X_train_a.shape[1])   # Initialize theta with zeros
threshold = 0.5
num_iter = 1000
cost = []
lr = 0.1
yl = y_train.shape[0]

for i in range(num_iter):
    h = sigmoid(X_train_a,theta)
    cost.append(cost_function(h,y_train))
    gradient = gradient_descent(X_train_a,h,y_train,yl)
    theta = update_loss(theta,lr,gradient)
outcome_r = predict(X_train_a,theta)

In [ ]:
################################# SLP with AnyLoss (Ours) #################################
def sigmoid(X,theta):
    z = np.dot(X,theta.T).astype(float)
    return 1.0/(1+np.exp(-z))

def sigmoid_2(p):   # Approximation Function
    s = (L*(p-threshold)).astype(float)
    return 1.0/(1+np.exp(-s))

############################# For AnyLoss ###############################

############### For fbeta score ###############
def cost_function_fbeta(bs,syh,syhy,bs_sy):
    loss = (1+bs)*syhy / ( bs_sy + syh )    # fbeta score
    return 1-loss

def gradient_descent_fbeta(p,X,yh,y,bs,syh,syhy,bs_sy):
    yp_pz = L*yh*(1-yh) * p*(1-p)
    return -((1+bs)*(np.dot(y*yp_pz*(bs_sy+syh), X)-np.dot(yp_pz*syhy, X))) / ((bs_sy + syh)**2)  # derivative of fbeta
############### For accuracy score ##############
def cost_function_acc(sy,syh,syhy,yl):
    loss = (yl-sy-syh+2*syhy)/yl   # accyracy
    return 1-loss

def gradient_descent_acc(p,X,yh,y,yl):   
    yp_pz = L*yh*(1-yh) * p*(1-p)
    return (np.dot(yp_pz, X) - 2*np.dot(y*yp_pz, X)) / yl    # derivative of accyracy
############### For geometric mean ###############
def cost_function_gmean(sy,syh,syhy,yl):
    loss = (syhy*(yl-syh-sy+syhy)/(sy*(yl-sy)))**0.5  # gmean
    return 1-loss

def gradient_descent_gmean(p,X,yh,y,sy,syh,syhy,yl):
    yp_pz = L*yh*(1-yh) * p*(1-p)
    repeat1 = np.dot(y*yp_pz, X)
    repeat2 = yl-syh-sy+syhy
    return -2*((repeat1*repeat2)+(-np.dot(yp_pz, X)+repeat1)*syhy)/(sy*(yl-sy)*syhy*(repeat2))**0.5  # derivative of gmean
############### For balanced accuracy ###############
def cost_function_balacc(sy,syh,syhy,yl):
    loss = (yl*(syhy+sy)-sy*(syh+sy)) / (2*sy*(yl-sy))     # balanced accuracy
    return 1-loss

def gradient_descent_balacc(p,X,yh,y,sy,yl):
    yp_pz = L*yh*(1-yh) * p*(1-p)
    return -(yl*np.dot(y*yp_pz, X)-sy*np.dot(yp_pz, X))/(2*sy*(yl-sy))  # derivative of balanced accuracy

#########################################################################

def update_loss(theta,learning_rate,gradient):
    return theta-(learning_rate*gradient)

def predict(X,theta):
    outcome = []
    result = sigmoid(X,theta)
    for i in range(X.shape[0]):
        if result[i] <= threshold:
            outcome.append(0)
        else:
            outcome.append(1)
    return outcome  


theta = np.zeros(X_train_a.shape[1])   # Initialize theta with zeros
threshold = 0.5
num_iter = 1000
L = 73
cost = []
lr = 0.1
sy = np.sum(y_train)
yl = y_train.shape[0]

for i in range(num_iter):
    p = sigmoid(X_train_a,theta)
    yh = sigmoid_2(p)           # Approximation (probablility -> like a label)
    syh = np.sum(yh)
    syhy = np.dot(yh,y_train)
    # For 'cost' and 'gradient', you can choose any pair of defined functions above for a specific goal.
    cost.append([   ])     # [   ]: cost_function_fbeta(bs,syh,syhy,bs_sy) // cost_function_acc(sy,syh,syhy,yl) // ...
    gradient = [   ]       # [   ]: gradient_descent_fbeta(p,X,yh,y,bs,syh,syhy,bs_sy) // ...
    theta = update_loss(theta,lr,gradient)
outcome_r = predict(X_train_a,theta)

## 2. Code in Multi-Layer Perceptron (MLP)

In [ ]:
################################ MSE ################################
def MSE(y_true, y_pred):
    return tf.reduce_mean(tf.math.square(y_true - y_pred))

################################ BCE ################################
def BCE(y_true, y_pred):
    return -tf.reduce_mean(y_true*tf.math.log(y_pred)+(1-y_true)*tf.math.log(1-y_pred))

################################ Ours_Accu ################################
def Ours_Accu(y_true, y_pred):
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    yl = y_train.shape[0]
    accu = (yl-tf.reduce_sum(y_true)-tf.reduce_sum(y_pred)+2*tf.reduce_sum(y_true*y_pred)) / yl
    return 1-accu

################################ Ours_Fbeta ################################
def Ours_Fbeta(y_true, y_pred):
#     beta = 1 
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    numerator = (1+beta**2)*tf.reduce_sum(y_true*y_pred)
    denominator = (beta**2)*tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    return 1-(numerator/denominator)

################################ Ours_Gmean ################################
def Ours_Gmean(y_true, y_pred):
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    syhy = tf.reduce_sum(y_true*y_pred)
    sy = tf.reduce_sum(y_true)
    yl = (y_train.shape[0])
#     gmean = syhy*(yl-tf.reduce_sum(y_pred)-sy+syhy)/(sy*(yl-sy))
    gmean = tf.sqrt(syhy*(yl-tf.reduce_sum(y_pred)-sy+syhy)/(sy*(yl-sy)))
    return 1-gmean

################################ Ours_BAccu ################################
def Ours_BAccu(y_true, y_pred):
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    syhy = tf.reduce_sum(y_true*y_pred)
    sy = tf.reduce_sum(y_true)
    yl = y_train.shape[0]
    baccu = (yl*(syhy+sy)-sy*(tf.reduce_sum(y_pred)+sy)) / (2*sy*(yl-sy))
    return 1-baccu

hidden_node = 2
kernel_initializer=keras.initializers.he_normal(seed=100)
activation = 'sigmoid'
learning_rate = 0.001
batch_size = int(X.shape[0]*0.9 * 0.05)
epochs=100
L = 73
threshold = 0.5

model = Sequential()
model.add(Dense(hidden_node, input_dim=X.shape[1], kernel_initializer=kernel_initializer))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(Dense(1))
opt = optimizers.Adam(learning_rate = learning_rate)
model.compile(loss=[  ], optimizer=opt, metrics=['accuracy']) # [  ]: MSE // BCE // Ours_Accu // Ours_Fbeta // ...
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch_size)
predicted = []
result = model.predict(X_test)
for i in range(X_test.shape[0]):
    if result[i] <= threshold:
        predicted.append(0)
    else:
        predicted.append(1)